In [1]:
import pandas as pd
import os

df = pd.read_csv("data/processed/clean_transactions.csv")
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

snapshot_date = df['invoice_date'].max()


In [2]:
rfm = df.groupby('customer_id').agg({
    'invoice_date': lambda x: (snapshot_date - x.max()).days,
    'invoice': 'nunique',
    'quantity': 'sum',
    'total_amount': 'sum'
}).reset_index()

rfm.columns = [
    'customer_id',
    'recency',
    'frequency',
    'total_units',
    'monetary_value'
]

rfm.head()


,customer_id,recency,frequency,total_units,monetary_value
0,12346.0,164,11,70,372.86
1,12347.0,2,2,828,1323.32
2,12348.0,73,1,373,222.16
3,12349.0,42,3,993,2671.14
4,12351.0,10,1,261,300.93


In [3]:
last_purchase = (
    df.groupby('customer_id')['invoice_date']
    .max()
    .reset_index()
)

last_purchase['days_since_last_purchase'] = (
    snapshot_date - last_purchase['invoice_date']
).dt.days

last_purchase['churn'] = (
    last_purchase['days_since_last_purchase'] > 90
).astype(int)

rfm = rfm.merge(
    last_purchase[['customer_id', 'churn']],
    on='customer_id'
)

rfm.head()


,customer_id,recency,frequency,total_units,monetary_value,churn
0,12346.0,164,11,70,372.86,1
1,12347.0,2,2,828,1323.32,0
2,12348.0,73,1,373,222.16,0
3,12349.0,42,3,993,2671.14,0
4,12351.0,10,1,261,300.93,0


In [4]:
os.makedirs("data/processed", exist_ok=True)

rfm.to_csv(
    "data/processed/customer_features_rfm.csv",
    index=False
)
